In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
epochs = 5 # 设置总共需要训练多少轮
batch_size_train = 32# 每次训练mini_batch的大小
batch_size_test = 100000 # 每次测试所用样本的大小
# learning_rate = 0.0011 # 学习率
# learning_rate = 0.0005
learning_rate = 0.01
log_interval = 256 # 按照mini_batch的大小打印测试结果
random_seed = 1 # 将随机种子设置为 1。可以根据需要选择任何整数值作为随机种子。
torch.manual_seed(random_seed) # 在使用相同的随机种子、相同的环境和相同的代码时，能保证得到相同的随机数序列。

In [3]:
# # 残差模块
# class ResidualBlock(torch.nn.Module):
#     #继承自ResidualBlock，传入输出通道数，输出通道数，滑动卷积核的步长
#     def __init__(self, in_channels, out_channels, stride=1):
#         super(ResidualBlock, self).__init__()
        
#         #采用3x3卷积核进行卷积，同时注意进行填充以保持图像尺寸不变
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
#                                 stride=stride, padding=1, bias=False)
#         #激活函数采用RELU
#         self.relu = nn.ReLU()
        
#     #正向传播，覆写torch.nn.Module中的魔法方法
#     def forward(self,x):
#         # 保存输入数据，采用恒等映射
#         identity = x
        
#         #进行卷积和激活
#         out =self.conv1(x)
        
#         out =self.relu(out)

#         # 还原结果
#         out = out + x
        
#         out = self.relu(out)
#         # 返回结果
#         return out
# # 初代
# # 构建包含ResidualBlock的CNN
# class ResNet_CNN(nn.Module):
#     def __init__(self,num_classes=10):
#         super(ResNet_CNN,self).__init__()
        
#         self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=1)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=1)
#         self.relu = nn.ReLU()

#         self.res1 = ResidualBlock(16, 16, stride=1)
#         self.res2 = ResidualBlock(32, 32, stride=1)
        
#         self.mp = nn.MaxPool2d(2)
        
#         # 用一个自适应均值池化层将每个通道维度变成1*1，此句可选
#         #self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        
#         self.fc = nn.Linear(800, 10)
#         # self.fc = nn.Linear(num_channels * height * width, num_classes)
#         # MINIST数据集中的图像，通常的高度和宽度是28x28像素。
#     def forward(self,x):
#         in_size = x.size(0)
 
#         x = self.mp(F.relu(self.conv1(x)))
#         x = self.res1(x)
#         x = self.mp(F.relu(self.conv2(x)))
#         x = self.res2(x)
 
#         x = x.view(in_size, -1)
#         x = self.fc(x)
#         return x


In [4]:
# # 
# class ResidualBlock(nn.Module):
#     # Residual Block需要保证输出和输入通道数x一样
#     def __init__(self, channels):
#         super(ResidualBlock, self).__init__()
#         self.channels = channels
#         # 3*3卷积核，保证图像大小不变将padding设为1
#         # 第一个卷积
#         self.conv1 = nn.Conv2d(channels, channels,
#                                kernel_size=3, padding=1)
#         # 第二个卷积
#         self.conv2 = nn.Conv2d(channels, channels,
#                                kernel_size=3, padding=1)
#         #第三个卷积，1*1
#         self.conv3 = nn.Conv2d(channels, channels, kernel_size=1)
        
#     def forward(self, x):
#         # 激活
#         y = F.relu(self.conv1(x))
#         y = self.conv2(y)
#         # 先求和 后激活
#         z = self.conv3(x)
#         return F.relu(z + y)
# # 构建包含ResidualBlock的CNN
# class ResNet_CNN(nn.Module):
#     def __init__(self):
#         super(ResNet_CNN, self).__init__()
#         #卷积层
#         self.conv1 = nn.Conv2d(1 ,32, kernel_size=3,padding=1)
#         self.conv2 = nn.Conv2d(32 ,64, kernel_size=3,padding=1)
#         self.conv3 = nn.Conv2d(64 ,32,kernel_size=3,padding=1)

#         #残差神经网络层，其中已经包含了relu
#         self.rblock1 = ResidualBlock(32)
#         self.rblock2 = ResidualBlock(64)
#         self.rblock3 = ResidualBlock(32)
        
#         #BN层，归一化，使数据在进行Relu之前不会因为数据过大而导致网络性能的不稳定
#         self.bn1 = nn.BatchNorm2d(32)
#         self.bn2 = nn.BatchNorm2d(64)
#         self.bn3 = nn.BatchNorm2d(32)
        
#         #最大池化，一般最大池化效果都比平均池化好些
#         self.mp = nn.MaxPool2d(2)
        
#         #fully connectected全连接层
#         self.fc1 = nn.Linear(32*3*3, 256)  # 线性
#         self.fc2 = nn.Linear(256, 64)
        
#         self.fc6 = nn.Linear(64, 10)  # 线性
#     def forward(self, x):
#         in_size = x.size(0)
        
#         x = self.conv1(x)   #channels:1-32  w*h:28*28  
#         # x = self.bn1(x)
#         x = self.mp(F.relu(x)) 
#         x = self.rblock1(x)
        
#         x = self.conv2(x)   #channels:32-64 w*h:14*14
#         x = self.mp(F.relu(x))
#         # x = self.bn2(x)
#         x = self.rblock2(x)
        
#         # x = self.mp(x)      #最大池化,channels:64-64    w*h:28*28->14*14
        
#         x = self.conv3(x)   #channels:64-32    w*h:7*7
#         # x = self.bn3(x)
#         x = self.mp(F.relu(x))
#         x = self.rblock3(x)
        
        
#         # x = self.mp(x)      #最大池化,channels:192-192  w*h:14*14->7*7

#         x = x.view(in_size, -1)     #展开成向量
#         x = F.relu(self.fc1(x))  # 使用relu函数来激活
#         x = F.relu(self.fc2(x))
#         return self.fc6(x)


In [5]:
# class ResidualBlock(nn.Module):
#     def __init__(self, channels):
#         super(ResidualBlock, self).__init__()
#         self.channels = channels
#         self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
 
#     def forward(self, x):
#         y = F.relu(self.conv1(x))
#         y = self.conv2(y)
#         return F.relu(x + y)
 
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3) # 88 = 24x3 + 16
 
#         self.rblock1 = ResidualBlock(16)
#         self.rblock2 = ResidualBlock(32)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.bn2 = nn.BatchNorm2d(32)
        
#         self.mp = nn.MaxPool2d(2)
#         self.fc_1 = nn.Linear(800, 64) # 暂时不知道1408咋能自动出来的
#         self.fc_2 = nn.Linear(64, 10)
 
#     def forward(self, x):
#         in_size = x.size(0)
 
#         x = self.mp(F.relu(self.conv1(x)))
#         x = self.rblock1(x)
#         x = self.bn1(x)
#         x = self.mp(F.relu(self.conv2(x)))
#         x = self.rblock2(x)
#         x = self.bn2(x)
#         x = x.view(in_size, -1)
#         x = self.fc_1(x)
#         x = self.fc_2(x)
#         return x


In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.cuda.empty_cache()
# model = Net().to(device)
# # model = ResNet_CNN().to(device)
# loss_f = nn.CrossEntropyLoss()# 自带softmax
# # loss_f = nn.NLLLoss()
# optimizer = optim.SGD(model.parameters(),lr=learning_rate,  momentum=0.5)
# # 使用StepLR策略
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [26]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.in_channels = 16
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = Net().to(device)
loss_f = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [27]:
# class ResidualBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1):
#         super(ResidualBlock, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(out_channels)
        
#         self.shortcut = nn.Sequential()
#         if stride != 1 or in_channels != out_channels:
#             self.shortcut = nn.Sequential(
#                 nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(out_channels)
#             )
        
#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         out += self.shortcut(x)
#         out = F.relu(out)
#         return out

# class Net(nn.Module):
#     def __init__(self, num_classes=10):
#         super(Net, self).__init__()
#         self.in_channels = 16
        
#         self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.layer1 = self._make_layer(16, 2, stride=1)
#         self.layer2 = self._make_layer(32, 2, stride=2)
#         self.layer3 = self._make_layer(64, 2, stride=2)
#         self.linear = nn.Linear(64, num_classes)

#     def _make_layer(self, out_channels, blocks, stride):
#         strides = [stride] + [1]*(blocks-1)
#         layers = []
#         for stride in strides:
#             layers.append(ResidualBlock(self.in_channels, out_channels, stride))
#             self.in_channels = out_channels
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.layer1(out)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = F.avg_pool2d(out, (out.size()[2], out.size()[3]))    # 修改此处
#         out = out.view(out.size(0), -1)
#         out = self.linear(out)
#         return out

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.cuda.empty_cache()
# model = Net().to(device)
# loss_f = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [28]:
# class ResidualBlock(nn.Module):
#     def __init__(self, channels):
#         super(ResidualBlock, self).__init__()
#         self.channels = channels
#         self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(channels) # Add BatchNorm layer
#         self.bn2 = nn.BatchNorm2d(channels) # Add BatchNorm layer
 
#     def forward(self, x):
#         y = F.relu(self.bn1(self.conv1(x)))
#         y = self.bn2(self.conv2(y))
#         return F.relu(x + y)
 
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3)

#         self.rblock1 = ResidualBlock(16)
#         self.rblock2 = ResidualBlock(32)

#         self.bn1 = nn.BatchNorm2d(16)
#         self.bn2 = nn.BatchNorm2d(32)

#         self.mp = nn.MaxPool2d(2)

#         self.dropout = nn.Dropout(p=0.5)

#         self.fc_1 = nn.Linear(800, 128) # Increase number of output features
#         self.fc_2 = nn.Linear(128, 64) # New hidden layer
#         self.fc_3 = nn.Linear(64, 10)
 
#     def forward(self, x):
#         in_size = x.size(0)

#         x = self.mp(F.relu(self.bn1(self.conv1(x))))
#         x = self.rblock1(x)
        
#         x = self.mp(F.relu(self.bn2(self.conv2(x))))
#         x = self.rblock2(x)
        
#         x = x.view(in_size, -1)
#         x = self.dropout(F.relu(self.fc_1(x)))

#         x = self.dropout(F.relu(self.fc_2(x)))
        
#         x = self.fc_3(x)
#         return x
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.cuda.empty_cache()
# model = Net().to(device)
# loss_f = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [29]:
# # 调用GPU
# torch.backends.cudnn.benchmark = True
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #print(device)
# torch.cuda.empty_cache()
# model = ResNet_CNN().to(device)
# model.to(device)

# # 构建损失函数
# loss_f = torch.nn.CrossEntropyLoss()


# # 构建优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)

# # 创建学习率调度器
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold=0.00005, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

# # 进行一次优化器的step操作
# optimizer.step()

# # 模拟一个评估指标（metrics）
# metrics = 0.8

# # 更新学习率调度器的状态
# scheduler.step(metrics)

# # 获取最新的学习率
# learning_rate = optimizer.param_groups[0]['lr']

In [30]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=True, download=True,
                                transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                (0.1307,), (0.3081,))
                                ])),
    batch_size=batch_size_train, shuffle=True)

In [31]:
# 训练模型
def train(epochs):
    for epoch in range(epochs):
        model.train() # 让模型进入训练模式
        for batch_idx,(data,target) in enumerate(train_loader):
            data,target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_f(output,target)
            
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                    epoch,batch_idx*len(data),len(train_loader.dataset),
                    100.*batch_idx/len(train_loader),loss.item()
                ))
        # 每个epoch结束后更新学习率
        scheduler.step()
                #epoch表示当前的训练轮数。
                #batch_idx*len(data)表示当前处理过的训练样本数量。
                #len(train_loader.dataset)是训练集中的总样本数。
                #100.*batch_idx/len(train_loader)计算了目前训练的进度（百分比形式）。
                #loss.item()则展示了最新批次的训练损失。

In [32]:

# def test():  # 定义一个名为`test`的函数，用于测试模型的性能
#     correct = 0  # 初始化正确预测的样本数量为0
#     total = 0  # 初始化测试集中总样本数量为0
#     model.eval()
#     with torch.no_grad():  # 设定PyTorch在此上下文中不计算梯度，这可以减小内存使用，加速计算，通常在评估模型时使用
#         for data in test_loader:  # 遍历测试集中的每一批数据
#             images, labels = data  # 将每一批数据分解为图像（输入）和标签（期望的输出）
#             images = images.to(device)
#             labels = labels.to(device)
#             outputs = model(images)  # 用模型对每一批图像进行预测
#             _, predicted = torch.max(outputs.data, dim=1)  # 从模型的输出中找出每个样本的最大值对应的类别，即预测的类别
#                                                            # 返回的是一个元组，其中第一个元素是每行的最大值组成的张量，第二个元素是每行最大值对应的列索引组成的张量。
#                                                            # max_values, max_indices = torch.max(predictions, dim=1)
#             total += labels.size(0)  # 更新测试样本的总数
#                                      # labels.size(0) 返回的是标签（labels）张量在第一个维度上的大小，也就是本批次（batch）中的样本数量。 ( N x c x w x h )
#             correct += (predicted == labels).sum().item()  # 如果预测的类别与真实标签相等，就将正确预测的样本数加1
#                                                            # 直接写 correct += (predicted == labels) 则会出错，因为这样 correct 会变成一个布尔值张量，
#                                                            # 需要先用 .sum() 来求布尔张量中 True 的数量，然后用 .item() 转换为标量才能进行累加操作。
#     print('accuracy on test set: %d %% ' % (100*correct/total))  # 计算并打印在测试集上的预测准确率


In [33]:
#把测试封装成函数
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images,labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data,dim=1)       #从第一维度开始搜索
            total += labels.size(0)
            correct += (predicted==labels).sum().item()

    return correct/total

In [34]:
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=False, download=True,
                                transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                (0.1307,), (0.3081,))
                                ])),
    batch_size=batch_size_test, shuffle=True)

In [ ]:
if __name__ == '__main__':
    epochs=1
    train(epochs)
    # test()
    print(test())

Train Epoch:0 [0/60000 (0%)]	Loss:0.018495


In [28]:
print(test())#$0.0001

0.9965


In [13]:
print(test())#$0.001

0.9909


In [13]:
print(test())#$0.0015

0.9913


In [30]:
# 保存模型 state_dict()是一个字典，保存了网络中所有的参数
# 转换并保存为torch.jit的模型
example_input = torch.rand(1, 1, 28, 28).to(device)  # 创建一个形状为 (1, 1, 28, 28) 的随机输入样本，并将其放置在指定的设备上（例如 GPU）
traced_model = torch.jit.trace(model, example_input)  # 使用 torch.jit.trace 函数对模型进行跟踪（tracing），生成跟踪模型（traced model）
torch.jit.save(traced_model, "traced_model_15.pt")  # 将跟踪模型保存到 "traced_model.pt" 文件中

In [17]:
# 保存模型 state_dict()是一个字典，保存了网络中所有的参数
# 转换并保存为torch.jit的模型
example_input = torch.rand(1, 1, 28, 28).to(device)  # 创建一个形状为 (1, 1, 28, 28) 的随机输入样本，并将其放置在指定的设备上（例如 GPU）
traced_model = torch.jit.trace(model, example_input)  # 使用 torch.jit.trace 函数对模型进行跟踪（tracing），生成跟踪模型（traced model）
torch.jit.save(traced_model, "traced_model_13.pt")  # 将跟踪模型保存到 "traced_model.pt" 文件中

"""
# 残差模块
class ResidualBlock(torch.nn.Module):
    #继承自ResidualBlock，传入输出通道数，输出通道数，滑动卷积核的步长
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

    #采用3x3卷积核进行卷积，同时注意进行填充以保持图像尺寸不变
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                            stride=stride, padding=1, bias=False)
   
    #激活函数采用RELU
    self.relu = nn.ReLU(可选参数)
   
    #正向传播
    def forward(self,x):
        # 保存输入数据，采用恒等映射
        identity = x
        
        #进行卷积和激活
        out =self.conv1(x)
        
        out =self.relu(out)

        
        # 还原结果
        删去了一句
        
        out = self.relu(out)
        # 返回结果
        return out
        
# 构建包含ResidualBlock的CNN
class ResNet_CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet_CNN,self).__init__()
        
        self.conv1 =nn.Conv2d(自行设计参数)
        
        
        self.relu = nn.ReLU()
        
        self.res2 = ResidualBlock(in_channels, out_channels, stride=1)
        
        # 用一个自适应均值池化层将每个通道维度变成1*1，此句可选
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        
        self.fc = nn.Linear(需要自己根据网络调整参数)
            
    def forward(self,x):
        x = self.conv1(x)
        
        x = self.relu(x)
        
        x = self.res2(x)
        
        # n个通道，每个通道1*1，输出n*1*1
        x = self.avg_pool(x)
        # 将数据拉成一维
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

SyntaxError: incomplete input (3793831218.py, line 7)

In [ ]:
# 初代
# 构建包含ResidualBlock的CNN
class ResNet_CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet_CNN,self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        
        self.relu = nn.ReLU()

        self.res1 = ResidualBlock(16, 16, stride=1)
        self.res2 = ResidualBlock(32, 32, stride=1)
        
        self.mp = nn.MaxPool2d(2)
        
        # 用一个自适应均值池化层将每个通道维度变成1*1，此句可选
        #self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        
        self.fc = nn.Linear(512, 10)
        # self.fc = nn.Linear(num_channels * height * width, num_classes)
        # MINIST数据集中的图像，通常的高度和宽度是28x28像素。
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.mp(x)
        x = self.res1(x)
        
        # n个通道，每个通道1*1，输出n*1*1
        #x = self.avg_pool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.mp(x)
        x = self.res2(x)
        

        # 将数据拉成一维
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [ ]:
# 78
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)  # 批归一化层
        self.relu = nn.ReLU(inplace=True)  # ReLU激活函数
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)  # 批归一化层
        self.stride = stride

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))  # 第一个卷积层 + 批归一化 + ReLU激活函数
        out = self.bn2(self.conv2(out))  # 第二个卷积层 + 批归一化
        out += self.shortcut(x)  # 残差连接
        out = self.relu(out)  # ReLU激活函数
        return out

class ResNet_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)  # 输入通道数为1，输出通道数为16的卷积层
        self.bn1 = nn.BatchNorm2d(16)  # 批归一化层
        self.relu = nn.ReLU()  # ReLU激活函数
        self.mp = nn.MaxPool2d(2)  # 最大池化层
        self.res1 = ResidualBlock(16, 16, stride=1)  # 第一个残差块

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)  # 输入通道数为16，输出通道数为32的卷积层
        self.bn2 = nn.BatchNorm2d(32)  # 批归一化层
        self.res2 = ResidualBlock(32, 32, stride=1)  # 第二个残差块

        self.fc = nn.Linear(512, num_classes)  # 全连接层，将特征展平后连接到输出类别数的神经元

    def forward(self, x):
        print(x.shape)
        out = self.conv1(x)  # 第一个卷积层
        out = self.bn1(out)  # 批归一化层
        out = self.relu(out)  # ReLU激活函数
        out = self.mp(out)  # 最大池化层
        out = self.res1(out)  # 第一个残差块
        print(x.shape)
        out = self.conv2(out)  # 第二个卷积层
        out = self.bn2(out)  # 批归一化层
        out = self.relu(out)  # ReLU激活函数
        out = self.mp(out)  # 最大池化层
        out = self.res2(out)  # 第二个残差块
        print(x.shape)
        out = out.view(out.size(0), -1)  # 将特征展平
        out = self.fc(out)  # 全连接层
        return out

In [ ]:
# 初代
# 构建包含ResidualBlock的CNN
class ResNet_CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet_CNN,self).__init__()
        
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=5)
        self.relu = nn.ReLU()

        self.res1 = ResidualBlock(8, 8, stride=1)
        self.res2 = ResidualBlock(16, 16, stride=1)
        self.res3 = ResidualBlock(32, 32, stride=1)
        
        self.mp = nn.MaxPool2d(2)
        
        # 用一个自适应均值池化层将每个通道维度变成1*1，此句可选
        #self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        
        self.fc = nn.Linear(512, 10)
        # self.fc = nn.Linear(num_channels * height * width, num_classes)
        # MINIST数据集中的图像，通常的高度和宽度是28x28像素。
    def forward(self,x):
        # 32x1x28x28
        x = self.conv1(x)
        x = self.relu(x)
        x = self.mp(x)
        x = self.res1(x)
        # 32x16x13x13
        # n个通道，每个通道1*1，输出n*1*1
        #x = self.avg_pool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        #x = self.mp(x)
        x = self.res2(x)
        # 32x64x5x5
        
        x = self.conv3(x)
        x = self.relu(x)
        #x = self.mp(x)
        x = self.res3(x)
        # 32x32x5x5
        # 将数据拉成一维
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [ ]:
# 残差模块
class ResidualBlock(torch.nn.Module):
    #继承自ResidualBlock，传入输出通道数，输出通道数，滑动卷积核的步长
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        #采用3x3卷积核进行卷积，同时注意进行填充以保持图像尺寸不变
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                                stride=stride, padding=1, bias=False)
        #激活函数采用RELU
        self.relu = nn.ReLU()
        
    #正向传播，覆写torch.nn.Module中的魔法方法
    def forward(self,x):
        # 保存输入数据，采用恒等映射
        identity = x
        
        #进行卷积和激活
        out =self.conv1(x)
        
        out =self.relu(out)

        
        # 还原结果
        out = out + x
        
        out = self.relu(out)
        # 返回结果
        return out
# 初代
# 构建包含ResidualBlock的CNN
class ResNet_CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet_CNN,self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5， paddle=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, paddle=1)
        self.conv3 = nn.Conv2d(channels, channels, kernel_size=1)
        self.relu = nn.ReLU()

        self.res1 = ResidualBlock(16, 16, stride=1)
        self.res2 = ResidualBlock(32, 32, stride=1)
        
        self.mp = nn.MaxPool2d(2)
        
        # 用一个自适应均值池化层将每个通道维度变成1*1，此句可选
        #self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        
        self.fc = nn.Linear(512, 10)
        # self.fc = nn.Linear(num_channels * height * width, num_classes)
        # MINIST数据集中的图像，通常的高度和宽度是28x28像素。
    def forward(self,x):
        in_size = x.size(0)
 
        x = self.mp(F.relu(self.conv1(x)))
        x = self.res1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.res2(x)
 
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x


In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 设计一个卷积神经网络，并在其中使用ResNet模块，在MNIST数据集上实现10分类手写体数字识别。
# 算一下每个数字的准确率
# 超参数
epochs = 10
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
# 这里的log_interval是指每隔多少个batch输出一次训练状态
log_interval = 10
random_seed = 1
# 设置种子，为了使得结果可复现
torch.manual_seed(random_seed)

# 从torchvision.datasets中加载MNIST数据集，并对数据进行标准化处理,参考网上
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   # 这里设置均值和方差的值
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=False, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_test, shuffle=True)


# 定义残差模块
class ResidualBlock(torch.nn.Module):
    # 这里stride是指卷积的步长,保持输入输出的维度不变
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        # padding为1，保证输入输出的维度不变,bias=False,因为后面有BN层
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        # 这里的inplace=True是指将ReLU的输出直接覆盖到输入中，可以节省的显存，但是会影响收敛性
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        # 这里的downsample是指如果输入输出的维度不一致，就需要对输入进行下采样，使得维度一致
        # 原理是使用1*1的卷积核对输入进行卷积，同时步长为stride，这样就可以保证输入输出的维度一致
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        # 保存输入数据，采用恒等映射
        identity = x

        # 第一个卷积层
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # 第二个卷积层
        out = self.conv2(out)
        out = self.bn2(out)

        # 下采样匹配卷积操作的输入输出维度
        identity = self.downsample(identity)

        # 还原结果
        out += identity
        out = self.relu(out)

        return out


# 构建包含ResidualBlock的网络，CNN
class ResNet_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet_CNN, self).__init__()
        # mnist是灰度图，所以输入通道为1，输出通道为16,卷积核为3，步长为1，padding为1说明让输入输出维度不变

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.res1 = ResidualBlock(16, 16)
        # 这里的stride=2,是因为输入输出维度不一致，需要下采样
        self.res2 = ResidualBlock(16, 32, stride=2)
        self.res3 = ResidualBlock(32, 64, stride=2)
        self.res4 = ResidualBlock(64, 128, stride=2)
        self.res5 = ResidualBlock(128, 128)
        self.res6 = ResidualBlock(128, 256)
        self.res7 = ResidualBlock(256, 256)

        # 用一个自适应均值池化层将每个通道维度变成1*1

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        # 感觉数字特征比较简单，一个全连接层就够了
        self.fc1 = nn.Linear(256, 7)
        self.fc2 = nn.Linear(7, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.res4(x)
        x = self.res5(x)
        x = self.res6(x)
        # x = self.res7(x)
        # 64个通道，每个通道1*1，输出64*1*1
        x = self.avg_pool(x)
        # 将数据拉成一维
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


# 实例化
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ResNet_CNN().to(device)
# 定义损失函数
loss_f = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# 训练模型
def train(epochs):
    for epoch in range(epochs):
        # 让BN层每一个mini-batch都要更新
        model.train()
        # 总损失
        # train_loss = 0
        # enumerate()函数用于将一个可遍历的数据对象组合为一个索引序列，同时列出数据和数据下标
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()
            # train_loss += loss.item()
            # 每个mini-batch打印一次,loss.item()是一个mini-batch的平均损失
            if batch_idx % log_interval == 0:
                print('Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()
                ))
    # 保存模型 state_dict()是一个字典，保存了网络中所有的参数
    # 转换并保存为torch.jit的模型
    example_input = torch.rand(1, 1, 28, 28).to(device)
    traced_model = torch.jit.trace(model, example_input)
    torch.jit.save(traced_model, "traced_model.pt")
    exit()


# 测试模型,每一个类别都要统计准确率，并统计总体准确率
def test():
    # 让BN层累计数据进行归一化
    # 读取模型
    model = torch.load('./traced_model.pt')
    # model.eval()
    # 总正确数
    correct_all = 0
    # 单个类别的正确数,这里用列表存储
    correct_class = list(0. for i in range(10))
    # 单个类别的总数
    total_class = list(0. for i in range(10))
    # 总测试数
    total_all = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            # torch.max()返回最大值和最大值的索引,这里要不要data?
            _, predicted = torch.max(output, dim=1)
            # 增加总测试数和总正确数
            total_all += labels.size(0)
            correct_all += (predicted == labels).sum().item()
            # 增加单个类别的总数和正确数
            for i in range(batch_size_test):
                label = labels[i]
                correct_class[label] += (predicted[i] == label).item()
                total_class[label] += 1
    # 打印每个类别的准确率
    for i in range(10):
        print('Accuracy of number{}:{:.2f}%'.format(
            i, 100 * correct_class[i] / total_class[i]
        ))
    # 打印总体准确率
    print('Accuracy of all:{:.2f}%'.format(100 * correct_all / total_all))


# main
if __name__ == '__main__':
    epochs = 10
    train(epochs)
    # test()
example_input = torch.rand(1, 1, 28, 28).to(device)  # 创建一个形状为 (1, 1, 28, 28) 的随机输入样本，并将其放置在指定的设备上（例如 GPU）
traced_model = torch.jit.trace(model, example_input)  # 使用 torch.jit.trace 函数对模型进行跟踪（tracing），生成跟踪模型（traced model）
torch.jit.save(traced_model, "traced_model_15.pt")  # 将跟踪模型保存到 "traced_model.pt" 文件中

Train Epoch:0 [0/60000 (0%)]	Loss:2.313058
Train Epoch:0 [640/60000 (1%)]	Loss:1.880683
Train Epoch:0 [1280/60000 (2%)]	Loss:1.147544
Train Epoch:0 [1920/60000 (3%)]	Loss:0.966166
Train Epoch:0 [2560/60000 (4%)]	Loss:0.925112
Train Epoch:0 [3200/60000 (5%)]	Loss:0.389216
Train Epoch:0 [3840/60000 (6%)]	Loss:0.627616
Train Epoch:0 [4480/60000 (7%)]	Loss:0.266529
Train Epoch:0 [5120/60000 (9%)]	Loss:0.128691
Train Epoch:0 [5760/60000 (10%)]	Loss:0.108642
Train Epoch:0 [6400/60000 (11%)]	Loss:0.214485
Train Epoch:0 [7040/60000 (12%)]	Loss:0.287485
Train Epoch:0 [7680/60000 (13%)]	Loss:0.125609
Train Epoch:0 [8320/60000 (14%)]	Loss:0.120873
Train Epoch:0 [8960/60000 (15%)]	Loss:0.280016
Train Epoch:0 [9600/60000 (16%)]	Loss:0.266129
Train Epoch:0 [10240/60000 (17%)]	Loss:0.132808
Train Epoch:0 [10880/60000 (18%)]	Loss:0.146302
Train Epoch:0 [11520/60000 (19%)]	Loss:0.200040
Train Epoch:0 [12160/60000 (20%)]	Loss:0.090015
Train Epoch:0 [12800/60000 (21%)]	Loss:0.139538
Train Epoch:0 [13440/